In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error,median_absolute_error
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict
from sklearn import metrics


import random

%matplotlib inline

In [2]:
path = '/Users/mjschillawski/Google Drive/Data/generalassembly/data/'
filename = 'Iowa_Liquor_Sales.csv'

## Load the data into a DataFrame
data = pd.read_csv(path+filename,usecols=['Date','Store Number','City', 
                     'Zip Code','County Number','County','Sale (Dollars)'])
                   
## Transform the dates if needed, e.g.
data["Date"] = pd.to_datetime(data["Date"])

/Users/mjschillawski/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
#subset 2016 data
#for predictions later
data2016 = data[np.logical_and(data['Date'] > '2015-12-31',data['Date'] < '2017-01-01')].copy()
print(data2016.shape)

(2279879, 7)


In [5]:
data2016.rename(columns={'Date':'date', 'Store Number':'storenum', 'City':'city', 
                     'Zip Code':'zipcode', 'County Number':'countynum', 
                     'County':'county',
                     'Sale (Dollars)':'sale_dollars'},inplace=True)

In [8]:
#Remove dollar signs and convert to numeric

bad_col = ['sale_dollars']

for item in bad_col:
    data2016[item] = data2016[item].str.replace('$','')
    data2016[item] = data2016[item].apply(pd.to_numeric)
    
data2016.head()

,date,storenum,city,zipcode,countynum,county,sale_dollars
8427861,2016-01-26,3621,SHELDON,51201,71.0,O'Brien,32.25
8428102,2016-01-12,2552,CEDAR RAPIDS,52404,57.0,Linn,58.74
8428103,2016-01-13,2513,IOWA CITY,52240,52.0,Johnson,352.44
8428104,2016-01-13,3869,CORALVILLE,52241,52.0,Johnson,88.11
8428105,2016-01-14,2106,CEDAR FALLS,50613,7.0,Black Hawk,29.37


In [10]:
#Annual 2016 totals by store
stores2016 = data2016.groupby('storenum')[['sale_dollars']].sum()
stores2016.head()

,sale_dollars
storenum,
2106,1117638.44
2113,76449.09
2130,937379.42
2152,13752.24
2178,247210.77


In [11]:
stores2016['sale_dollars'].sum()

251531858.14999935

In [12]:
data2016['sale_dollars'].sum()

251531858.15018967

In [13]:
data2016['date'].describe()

count                 2279879
unique                    246
top       2016-06-28 00:00:00
freq                    15107
first     2016-01-04 00:00:00
last      2016-12-30 00:00:00
Name: date, dtype: object